# Conférences Python Master TIDE #6

**Dataviz avec plotly express**

1. **Réduction de dimensions**
2. **Cartes proportionnelles**
3. **Cartes géographiques**
4. **Divers**


&copy; 2025 Francis Wolinski

In [ ]:
#!pip install plotly

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px

Il peut être nécessaire de fixer le moteur de rendu de **plotly** avec l'instruction suivante en en choisissant un parmi la liste fournie.

In [ ]:
# renderers
import plotly.io as pio
print(pio.renderers)
pio.renderers.default = 'jupyterlab'

Les graphiques sont produits avec une barre d'outils offrant différentes fonctions (snapshot, zoom, sélection, recentrage...).

Il est possible de masquer ce menu avec l'instruction :
```python
fig.show(config={'displayModeBar': False})
```

Les options de configuration sont explicitée dans cette page : https://plotly.com/python/configuration-options/

#### Correspondances matplotlib / seaborn / plotly (extrait)

matplotlib|seaborn|plotly
-|-|-
bar|barplot|bar
pie|barplot|pie
-|stripplot|strip
scatter|scatterplot|scatter
boxplot|boxplot|box
violinplot|violinplot|violin
hist|displot|histogram
-|pairplot|scatter_matrix
-|joinplot|density_heatmap
-|FacetGrid|paramètres "facet_row" et "facet_col"

### 1. Réduction de dimensions

L'objectif de cette partie est de mettre en oeuvre des algorithmes de réduction de dimension pour visualiser en 2D des données numériques multi-dimensionnelles.

Le jeu de données vient des Nations Unies (UNCTAD) sur les facteurs de développement du e-commerce par pays :
- \% d'individus utilisant Internet, source International Telecommunication Union (ITU)
- \% d'individus avec un compte bancaire (15+), source World Bank
- nombre de serveurs Internet sécurisés par 1 million de personnes, source World Bank
- score de fiabilité postale, source Universal Postal Union (UPU)

Source : https://unctad.org/system/files/official-document/tn_unctad_ict4d17_en.pdf

**Exercice : Préparation des données**
- Charger le jeu de données de e-commerce "./data/e-commerce_index_2020.txt"
- Charger également la page "https://www.geonames.org/countries/" en sélectionnant le DataFrame avec ~250 lignes, ou bien le fichier "./data/GeoNames.html". Attention aux *NA* !
- Effectuer une jointure *inner* entre le dataset e-commerce et les pays. Le résultat a combien de lignes ?
- Utiliser le dictionnaire *mapping* pour transformer la colonne "Economy" du dataset e-commerce et refaites la jointure. Le résultat a combien de lignes ?

In [ ]:
# mapping entre les noms des pays
mapping = {
    'Bolivia (Plurinational State of)': 'Bolivia',
    'China, Hong Kong SAR': 'Hong Kong',
    'Congo': 'Congo Republic',
    'Cōte d\'Ivoire': 'Ivory Coast',
    'Dem. Rep. of the Congo': 'DR Congo',
    'Iran (Islamic Republic of)': 'Iran',
    'Korea, Republic of': 'North Korea',
    'Lao People\'s Dem. Rep.': 'Laos',
    'Netherlands': 'The Netherlands',
    'Republic of Moldova': 'Moldova',
    'Russian Federation': 'Russia',
    'Syrian Arab Republic': 'Syria',
    'Turkey': 'Türkiye',
    'United Republic of Tanzania': 'Tanzania',
    'United States of America': 'United States',
    'Venezuela (Bolivarian Rep. of)': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'Yemen, Arab Republic': 'Yemen',
}

len(mapping)

**Exercice : Réduction de dimension**

* Sélectionner les données des 4 facteurs de e-commerce.
* Normaliser ces données avec un StandardScaler (dans `sklearn.preprocessing`) avec la méthode `fit_transform()`. On obtient une matrice à 4 colonnes.
* Instancier un `TSNE` (dans `sklearn.manifold`) avec le paramètre `perplexity=8` (lié au nombre de voisins utilisés par l'algorithme), et un `random_state` pour la reproductibilité des résultats.
* Appliquer la méthode `fit_transform()` aux données standardisées. On obtient une matrice à 2 colonnes.
* Fusionner les 2 colonnes obtenues transformées en Series avec les colonnes "Population", "Continent" et "Country" des pays avec la fonction `concat()`. On obtient un DataFrame à cinq colonnes.

**Exercice : Affichage**

* Compléter le code ci-dessous pour afficher le résultat.

In [ ]:
# à vous de jouer
fig = px.scatter(data_frame=___,
           x=___,
           y=___,
           color=___,
           size=___,
           text=___,
           color_discrete_sequence = px.colors.qualitative.Set1,
           size_max=40)

fig.update_layout(font_size=10,
                  font_family="Calibri",
                  showlegend=False,
                  height=600,
                  plot_bgcolor="white")

**Exercice : Analyse en composantes principales**

* A l'étape 2 instancier un PCA (dans `sklearn.decomposition`) avec le paramètre `n_components=2` et remplacez le TSNE par un PCA.
* Obtenir les composantes de l'ACP (`pca.components_`) ainsi que le % de variance expliquée par composante (`pca.explained_variance_ratio_`)
* Au moment de la fusion des résultats, il faudra peut-être prendre l'opposé de la première colonne numérique, afin d'obtenir un graphique standard avec, à droite, les pays dont les facteurs de e-commerce sont les plus élevés.
* Afficher le résultat.
* Voir également le lien : https://yotta-conseil.fr/Developing_an_Intuition_on_PCA.html

### 2. Cartes proportionnelles

Une carte proportionnelle *treemap* (resp. *sunburst*) est une représentation rectangulaire (resp. circulaire) de données hiérarchiques dans un espace limité.

In [ ]:
# dataset
fortune = pd.read_csv("./data/Fortune_1000.csv", na_values="-")
fortune = fortune.fillna(0)
fortune.head()

In [ ]:
# treemap
px.treemap(fortune,
          path=["sector", "company"],
          values="Market Cap")

**Exercice**

1. *sunburst* avec les 4 plus grandes capitalisations boursières par secteur d'activité.
2. *treemap* avec la hiérarchie à 3 niveaux : état, ville, société avec le nombre d'employés.
3. A partir du fichier "./data/correspondance-code-insee-code-postal.csv", produire une *treemap* avec la hiérarchie à 3 niveaux : Région, Département, Commune avec la Population limitée aux 3 villes les plus peuplées de chaque département.

### 3. Cartes choroplèthes

Une carte choroplèthe est une carte thématique où les régions sont colorées ou remplies d'un motif qui montre une mesure statistique.

Pour produire une carte choroplèthe, il faut récupérer un fichier *geojson* de la partie du monde considérée. Ce fichier contient entre autres 2 informations importantes :
- Une clé qui désigne chaque sous-région,
- La description d'un polygone ou d'un multi-polygone sous la forme d'une liste de coordonnées (latitude et longitude),

Il faut ensuite faire coïncider les valeurs de la clé du fichier *geojson* avec celles de la colonne du DataFrame qui contient les données relatives à chaque sous-région.

Il est possible ensuite d'utiliser différents types de projections et différents fonds de cartes.

Le fichier "./data/departements.geojson" utilisé ci-après provient du site : https://france-geojson.gregoiredavid.fr/ et il faut analyser le fichier pour trouver la clé correspondant à chaque sous-région.

**Exercice**

Charger le fichier "./data/departements.geojson" avec la librairie **json** dans une variable *departements* qui est un *dict* Python :
- analyser `departements["features"]`,
- sélectionner le premier élément de `departements["features"]` qui est une *list* Python et l'analyser.

Charger le fichier "./data/correspondance-code-insee-code-postal.csv" dans un DataFrame :
- comparer la colonne "Code Département" du DataFrame avec les valeurs accédées par les clés 'properties' et 'code' des éléments de `departements["features"]`,
- produire un DataFrame *df* avec pour chaque "Code Département" la population totale.

Produire la carte directement en complétant les informations manquantes :
- *data_frame* : le DataFrame,
- *locations* : la colonne avec la clé d'accès dans le DataFrame,
- *color* : la colonne avec la grandeur représentée dans le DataFrame,
- *geojson* : le dictionnaire issu du fichier *geojson*,
- *featureidkey* : les clés d'accès dans le fichier *geojson* (sous la forme d'une chaîne où les clés sont séparées par des `.`)

Les autres informations permettent de paramétrer la carte désirée : nuancier de couleurs, type de carte, zoom, centrage, etc.

Produire également une carte choroplèthe avec la moyenne de l'Altitude Moyenne de chaque département, en utilisant le fond de carte "satellite" et le nuancier "reds".

In [ ]:
# choropleth_map
px.choropleth_map(data_frame=__,
                  locations=__,
                  color=__,
                  geojson=__,
                  featureidkey=__,
                  color_continuous_scale="teal",
                  map_style="carto-positron",
                  zoom=4.0,
                  center = {"lat": 47.0, "lon": 0.0},
                  opacity=0.5,
                  labels={'Population': 'Population en milliers'},
                  width=800,
                  height=600)


### 4. Divers

La librairie **plotly** permet également de produire des cartes avec un nuage de points.

#### 4.1 Carte avec nuage de points

In [ ]:
# scatter_map
geo = pd.read_csv("data/correspondance-code-insee-code-postal.csv",
                  sep=";",
                  index_col="Code INSEE",
                  usecols=range(11))

geo[["Latitude", "Longitude"]] = geo["geo_point_2d"].str.extract(r"(.*), (.*)").astype(float)
statut_cat = pd.CategoricalDtype(categories=['Commune simple', 'Sous-préfecture', 'Chef-lieu canton', 'Préfecture', 'Préfecture de région', "Capitale d'état"],
                                 ordered=True)
geo["Statut"] = geo["Statut"].astype(statut_cat)

# on sélection les villes au moins Préfecture
df = geo.loc[geo["Statut"]>="Préfecture"]

px.scatter_map(data_frame=df,
                 lat="Latitude",
                 lon="Longitude",
                 hover_name="Commune",
                 size="Population",
                 color="Population",
                 color_continuous_scale="amp",
                 map_style="carto-positron",
                 zoom=4,
                 center={"lat": 47.0, "lon": 3.0},
                 labels={"Population": "Population en millier"},
                 width=700,
                 height=500)

#### 4.2 Diagramme de Sankey

Un diagramme de Sankey est un type de diagramme de flux dans lequel la largeur des flèches est proportionnelle au flux représenté.

Pour ce type de diagramme, il faut utiliser le module **graph_objects** de la librairie.

Il faut établir la liste unicisée des labels des noeuds à représenter et ensuite produire 3 listes alignées correspondant aux arêtes du graphique :
- *source* : indices des noeuds de départ dans la liste des labels,
- *target* : indices des noeuds d'arrivée dans la liste des labels,
- *value* : valeurs à représenter.

In [ ]:
# dataset fortune
fortune[["sector", "state", "company", "Market Cap"]].head()

In [ ]:
# on sélectionne les 4 plus grandes capitalisations des 5 secteurs les plus importants

secteurs = fortune.groupby("sector")["Market Cap"].sum().nlargest(5).index

fortune2 = (fortune
            .loc[fortune["sector"].isin(secteurs)]
            .groupby(["sector"], as_index=False)[["sector", "state", "company", "Market Cap"]]
            .apply(lambda g: g.nlargest(4, "Market Cap"))
            .sort_values(["sector", "state", "Market Cap"], ascending=[True, True, False])
            .reset_index(drop=True)
           )
fortune2

In [ ]:
# on établit la liste des labels uniques des noeuds à représenter
labels = list(fortune2["sector"].unique()) + list(fortune2["state"].unique())
labels

In [ ]:
# on produit le diagramme de Sankey
# tout se passe dans la constitution du dictionnaire mis dans l'argument "link"

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = [labels.index(x) for x in fortune2["sector"]],  # indices des noeuds de départ dans la liste des labels
      target = [labels.index(x) for x in fortune2["state"]],  # indices des noeuds d’arrivée dans la liste des labels
      value = fortune2["Market Cap"]  # valeurs à représenter
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10, height=400)
fig.show()

**Exercice avec le PLF 2025**

- Charger le fichier "./data/plf25-recettes-du-budget-general.csv" et faire un diagramme de Sankey avec les colonnes "Type de recettes", "Libellé" et "Montant Recettes PLF". Pour simplifier le graphique, il est possible de passer les libellés à "Divers" lorque le montant est inférieur à 1 Milliard (utiliser un `.combine()` avec les colonnes et la fonction adéquates).
- Charger le fichier "./data/plf25-depenses-du-budget-general.csv" et faire un diagramme de Sankey avec les colonnes "Ministère", "Mission" et "AE  PLF". Pour simplifier le graphique, il est possible de sommer les montants à ministère et mission identiques (utiliser un `.groupby()`) avec les colonnes et la fonction adéquates).

Autre exemple : https://huguesg.fr/site/index.php/2024/01/13/visualisation-du-budget-de-letat/